In [1]:
!pip install yfinance

In [184]:
import numpy as np
import pandas as pd
import yfinance as yf

In [83]:
#ALGORITHM OUTLINE
# (1) Find current_price for stock of interest (use stocks most recent close price)
# (2) Check if .93*current_price \in [low,high] for stock of interest for the last week (last five days)

In [70]:
# algorithm for single date
# dat = yf.Ticker("^TASI.SR")
# history_for_lastweek_5d = dat.history(period='5d')
# low = history_for_lastweek_5d["Low"].min()
# high = history_for_lastweek_5d["High"].max()
# current_price = history_for_lastweek_5d["Close"][0]
# price_of_interest = .93*current_price
# price_of_interest_within_last_week = low <= price_of_interest and price_of_interest <= high

<ipython-input-70-2770be6dda2d>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = history_for_lastweek_5d["Close"][0]


In [288]:
# algorithm for every date between start date and end date
data = yf.download("^TASI.SR", start="2010-01-01", end="2025-01-23")

data["prv_low_1"] = data["Low"].shift(1)
data["prv_low_2"] = data["Low"].shift(2)
data["prv_low_3"] = data["Low"].shift(3)
data["prv_low_4"] = data["Low"].shift(4)
data["prv_low_5"] = data["Low"].shift(5)
data["Low_Last_5d"] = data[["prv_low_1", "prv_low_2","prv_low_3","prv_low_4","prv_low_5"]].values.min(1)
data = data.drop(["prv_low_1", "prv_low_2", "prv_low_3", "prv_low_4", "prv_low_5"], axis=1)

data['prv_high_1'] = data["High"].shift(1)
data['prv_high_2'] = data["High"].shift(2)
data['prv_high_3'] = data["High"].shift(3)
data['prv_high_4'] = data["High"].shift(4)
data['prv_high_5'] = data["High"].shift(5)
data["High_Last_5d"] = data[["prv_high_1", "prv_high_2","prv_high_3","prv_high_4","prv_high_5"]].values.max(1)
data = data.drop(["prv_high_1", "prv_high_2", "prv_high_3", "prv_high_4", "prv_high_5"], axis=1)

data["seven_percent_of_close_price"] = .93*data["Close"]
data["Close"] = 1*data["Close"]
data["seven_percent_of_close_price_within_last_five_days"] = np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False)
data["Last_Close"] = data["Close"].shift(1)
data["close_diff"] = data["Close"].values.reshape(data.shape[0]) - data["Last_Close"].values.reshape(data.shape[0])
data["up_from_yesterday"] = np.where((data["close_diff"] > 0), True, False)
data["buy"] = data["seven_percent_of_close_price_within_last_five_days"] & data["up_from_yesterday"]
data = data.drop(["Last_Close","close_diff","seven_percent_of_close_price"], axis=1)
data.head(15)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Low_Last_5d,High_Last_5d,seven_percent_of_close_price_within_last_five_days,up_from_yesterday,buy
Ticker,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,^TASI.SR,,,,,
Date,,,,,,,,,,
2010-01-04,6201.759766,6202.319824,6134.720215,6149.910156,80335000,NaN,NaN,False,False,False
2010-01-05,6239.100098,6254.029785,6201.759766,6201.759766,102019700,NaN,NaN,False,True,False
2010-01-06,6260.899902,6269.810059,6232.709961,6239.100098,107294800,NaN,NaN,False,True,False
2010-01-07,6260.899902,6260.899902,6260.899902,6260.899902,0,NaN,NaN,False,False,False
2010-01-11,6291.390137,6318.140137,6248.620117,6318.140137,103442400,NaN,NaN,False,True,False
2010-01-12,6266.259766,6291.390137,6238.649902,6291.390137,139063700,6134.720215,6318.140137,False,False,False
2010-01-13,6262.830078,6304.049805,6242.209961,6266.259766,185619400,6201.759766,6318.140137,False,False,False
2010-01-14,6262.830078,6262.830078,6262.830078,6262.830078,0,6232.709961,6318.140137,False,False,False


In [225]:
print(np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False))
print(np.where((data['Low_Last_5d'] <= data['seven_percent_of_close_price']) & (data['seven_percent_of_close_price'] <= data['High_Last_5d']), True, False).shape)

[False False False ... False False False]
(3295,)


In [249]:
print(data["Low"].values.reshape(3295,1).shape)
print(data["Low_Last_5d"].values.reshape(3295,1).shape)

(3295, 1)
(3295, 1)


In [235]:
data['Last_Close'] = data['Last_Close'].values
print(data['Last_Close'].shape)

(3295,)


In [244]:
print(data["Close"].shape)
print(data["Last_Close"].shape)


(3295, 1)
(3295,)


In [256]:
data["Low_Last_5d"] = data["Low_Last_5d"].values.reshape(3295)

In [261]:
data["Low_Last_5d"].values.reshape(3295).shape

(3295,)

In [262]:
data["Low"].values.reshape(3295).shape

(3295,)

In [270]:
data["close_diff"] = data["Close"].values.reshape(3295) - data["Last_Close"].values.reshape(3295)

In [277]:
data["up_from_yesterday"].sum()

1711

In [275]:
data["up_from_yesterday"].sum()

1711

In [287]:
data["buy"].sum()

53